In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
import timeit

train=pd.read_csv("train_sales.csv", header=0)
test=pd.read_csv("test_sales.csv", header=0)
train=train[train['unit_sales']>=0]
test=test[test['unit_sales']>=0]

train=train[train['unit_sales']>0]
test=test[test['unit_sales']>0]

In [2]:
del train['Unnamed: 0']
del test['Unnamed: 0']
train['full_date']=pd.to_datetime(train['full_date'])
test['full_date']=pd.to_datetime(test['full_date'])
train['day_of_week']=train['full_date'].dt.dayofweek
test['day_of_week']=test['full_date'].dt.dayofweek
def replace_nan(x):
    if x!=True:
        x=False
    return x
train['onpromotion']=train['onpromotion'].map(replace_nan)
test['onpromotion']=test['onpromotion'].map(replace_nan)

del train['full_date']
del test['full_date']
del train['id']
del test['id']

In [3]:
dum_dums_train=pd.get_dummies(train[['month', 'quarter', 'type', 'cluster', 'family', 'class', 'store_nbr', 'day_of_week']])
train=pd.concat([train[['onpromotion', 'year', 'holiday', 'transactions', 'perishable', 'unit_sales', 'item_nbr']], dum_dums_train], axis=1)
dum_dums_test=pd.get_dummies(test[['month', 'quarter', 'type', 'cluster', 'family', 'class', 'store_nbr', 'day_of_week']])
test=pd.concat([test[['onpromotion', 'year', 'holiday', 'transactions', 'perishable', 'unit_sales', 'item_nbr']], dum_dums_test], axis=1)

In [4]:
test_items=test['item_nbr'].unique().tolist()
train_items=train['item_nbr'].unique().tolist()

test_only=list(set(test_items)-set(train_items))
train_only=list(set(train_items)-set(test_items))

predict=set(test_items) & set(train_items)

print "items in the test set that are not in the training set: {}".format(test_only)
print "there are {} items in both the training and validation set".format(len(predict))

items in the test set that are not in the training set: [2116139, 2122947]
there are 4014 items in both the training and validation set


In [5]:
avg_val=train['unit_sales'].mean()

avg_val

8.554771178160252

In [6]:
6*[avg_val]

[8.554771178160252,
 8.554771178160252,
 8.554771178160252,
 8.554771178160252,
 8.554771178160252,
 8.554771178160252]

In [7]:
from sklearn.model_selection import (train_test_split,KFold)
from sklearn.metrics import mean_squared_error
from sklearn.neural_network import MLPRegressor
import math

In [8]:
#line below for testing code

train_save=train.copy(deep=True)
test_save=test.copy(deep=True)

mlp2 = MLPRegressor(hidden_layer_sizes=(2,), activation = 'tanh', learning_rate='adaptive', 
                    random_state=12, batch_size=200)

perishable=[]
unit_sales=[]
unit_sales_pred=[]
#item_nbr=[]

n=0
start=timeit.default_timer()

for item in predict:
    
    n=n+1
    
    if n%100==1:
        print n
    
    
    train_sub=train_save[train_save['item_nbr']==item]
    test_sub=test_save[test_save['item_nbr']==item]
    y_test_sub=np.log(test_sub['unit_sales']+1)
    y_train_sub=np.log(train_sub['unit_sales']+1)
    del train_sub['unit_sales']
    del test_sub['unit_sales']
    
    # Fitting on both X's and Y's
    sub_mod = mlp2.fit(train_sub, y_train_sub)
    sub_pred=sub_mod.predict(test_sub)
    
    perishable+=test_sub['perishable'].tolist()
    unit_sales+=y_test_sub.tolist()
    unit_sales_pred+=sub_pred.tolist()
    

elapsed=timeit.default_timer()-start

print elapsed
###don't forget the items that are only in the test set

1


/Users/brettscroggins/anaconda/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)
/Users/brettscroggins/anaconda/lib/python2.7/site-packages/sklearn/neural_network/multilayer_perceptron.py:357: UserWarning: Got `batch_size` less than 1 or larger than sample size. It is going to be clipped
  warnings.warn("Got `batch_size` less than 1 or larger than "


101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
1248.72709918


In [9]:
test_only=list(set(test_items)-set(train_items))

for item in test_only:
    
    train_sub=train_save[train_save['item_nbr']==item]
    test_sub=test_save[test_save['item_nbr']==item]
    y_test_sub=np.log(test_sub['unit_sales']+1)
    y_train_sub=np.log(train_sub['unit_sales']+1)
    del train_sub['unit_sales']
    del test_sub['unit_sales']
    
    #sub_pred=len(y_test_sub)*[avg_val]
    perishable_list=len(y_test_sub)*[test_sub['perishable'].values[0]]
    
    perishable+=perishable_list
    unit_sales+=y_test_sub.tolist()
    unit_sales_pred+=len(y_test_sub)*[np.log(avg_val+1)]
    
print len(perishable), len(unit_sales), len(unit_sales_pred)

2514263 2514263 2514263


In [10]:
score=pd.DataFrame({'perishable': perishable, 'unit_sales': unit_sales, 'unit_sales_pred': unit_sales_pred})
score['weight']=score['perishable']*0.25+1
score['num']=score['weight']
score['num']=score['num']*(score['unit_sales_pred']-score['unit_sales'])**2
num=sum(score['num'])
den=sum(score['weight'])
nwrmsle=(float(num)/den)**0.5



nwrmsle

#len(score)

0.78284487415326287